# Creating a conditional chat 
*The goal here is to make a chat that can fill out a for utilizing a chat with a user*

First step is to download the needed packages

In [1]:
import openai
import pandas as pd
from datetime import datetime
import json
import os
from string import Template
import time

### Set up your key parameters
Response limiting is to avoid over requesting the api if you are utilizing the trial version of the api

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')
model = "gpt-3.5-turbo-0613"
response_limit = 4
response_number = 1

### Generic model completion function with functionality to support a function call

In [3]:
def get_model_completion(
    messages, 
    functions = None, 
    function_call = "none", 
    response_limit = response_limit, 
    response_number = response_number):
    if response_number % response_limit == 0:
        time.sleep(30)
    if functions == None:
        response = openai.ChatCompletion.create(
            model = model,
            messages = messages
        )
    else:
        response = openai.ChatCompletion.create(
            model = model,
            messages = messages,
            functions = functions,
            function_call = function_call,
            temperature = 0
        )
    message = response["choices"][0]["message"]
    if message.get("function_call"):
        return json.loads(message["function_call"]["arguments"])
    else:
        return message["content"]

### Helper functions to support the chain and handle missing fields

In [4]:
question_formation_prompt = """Your task is to ask a user for information about the following fields delimited by \
triple back ticks.
``` 
${fields}
```
"""

def key_search(key, func):
    if type(func) == str:
        t = 0
    elif type(func) == list:
        for i in func:
            key_search(key, i)
    elif key not in func.keys():
        for f in func.keys():
            val = key_search(key, func[f])
            if val != None:
                return val
    else:
        return func[key]['description']

### Function to run the chain

In [5]:
def function_response_chain(
    starting_prompt : str, 
    functions : list,
    function_name : str, 
    needed_fields : list, 
    response_number = response_number) -> dict:
    complete = False
    messages = [
        {
            "role" : "user",
            "content" : starting_prompt
        }
    ]
    f_id = 0
    for idx, f in enumerate(functions):
        if f.get("name") == function_name:
            f_id = idx
    while not complete:
        function_call = {"name" : function_name}
        response = get_model_completion(messages, functions, function_call, response_number = response_number)
        response_number += 1
        missing = []
        for i in needed_fields:
            if i not in list(response.keys()):
                missing.append(i)
            elif response[i] == '-1':
                missing.append(i)
        if len(missing) == 0:
            complete = True
            return response
        fields = []
        for m in missing:
            desc = key_search(m, functions[f_id])
            fields.append(f"{m} : {desc}")
            
        fields = "\n".join(fields)
        prompt_template = Template(question_formation_prompt)
        q_prompt = prompt_template.substitute(fields = fields)
        q_message = [
            {
                "role" : "system",
                "content" : q_prompt
            },
            {
                "role" : "user",
                "content" : "What do you need to know"
            }
        ]
        q_response = get_model_completion(q_message, response_number = response_number)
        response_number += 1
        in_ = input(f"Model: {q_response} \n\nUser Input: ")
        print('\n')
        messages.append({
            "role" : "user",
            "content" : in_
        })

### Function structure
Important notes:
1. Only include fields you are ok with the model guessing on in required
2. Fields that you are not ok with guesses make it `if not provided return -1`
3. For enums if this is not a field you are ok with a guess include -1 in the enum values. Make sure that it is the last value

In [6]:
functions = [
    {
        "name" : "jira_api_call",
        "description" : """This is a wrapper on the jira api that takes content from the user and writes /
        to their jira environment""",
        "parameters" : {
            "type" : "object",
            "properties" : {
                "title" : {
                    "type" : "string",
                    "description" : """This is the headline title of the jira story. This should describe the \
                    story in about 5 to 10 words"""
                },
                "description" : {
                    "type" : "string",
                    "description" : """This is the series of instructions that the engineering team will use \
                    to determine the scope of the story and how to complete the work to implement it"""
                },
                "owner" : {
                    "type" : "string",
                    "description" : """This is the product manager that will own this task. Do not guess. Do not \
                    make something up. If not provided return -1.""",
                    "enum" : ["Nelson Strimple", "Taina Felix", "LeBron James", "-1"]
                },
                "deadline" : {
                    "type" : "string",
                    "description" : f"""This is the date that the project has to be completed by. It has \
                    to be after {datetime.now().date()}. It is to be formated year-month-day. If not provided \
                    return -1"""
                }
            },
            "required" : ["title", "description"]
        }
    }
]

# Running a prompt that will leave out deadline and owner on purpose
test_prompt = """
You are product management assistant. Your role is to chat about new features /
or products that they want to produce.  You will then format that information to use the function /
jira_api_call. The user input will be delimited by triple back ticks.

```
Create a new api endpoint that will take in user input, call a large language, and then return /
the model output to a user.
```
"""

elements = ["title", "description", "owner", "deadline"]

In [7]:
res = function_response_chain(test_prompt, functions, "jira_api_call", elements)

Model: I need to know the following information:

- Who will be the owner of this task? 
- What is the deadline for completing the project? 

User Input: Taina will be the owner


Model: Please provide the deadline for the project. 

User Input: End of September




In [8]:
print(json.dumps(res, indent = 1))

{
 "title": "Create a new API endpoint for language translation",
 "description": "Develop a new API endpoint that will take in user input, call a large language model, and return the model output to the user.",
 "owner": "Taina Felix",
 "deadline": "2023-09-30"
}
